# Import Libray

In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import joblib
import pickle

# Load Data

In [ ]:
!git clone https://github.com/Hafiyyanrevi/Drug-Side-Effect-Prediction-GSA-SVM.git data_TA

Cloning into 'data_TA'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 165 (delta 49), reused 63 (delta 21), pack-reused 64 (from 1)
Receiving objects: 100% (165/165), 659.56 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [ ]:
dataset = pd.read_csv('data_TA/data/processed_data/smiles_dataset.csv')

In [ ]:
dataset

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,Eye disorders
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1084,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1085,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1086,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


# Data Exploration

In [ ]:
value_counts = dataset['Eye disorders'].value_counts()
print("perbandingan 0 & 1 =", value_counts[0], "/", value_counts[1])

perbandingan 0 & 1 = 401 / 687


**Ensure there are no duplicate rows**

In [ ]:
duplicate_rows = dataset.duplicated().sum()
print("baris duplikat =", duplicate_rows)

baris duplikat = 0


# Data Splitting

**Take all columns except the last column as features & the last column as label**

In [ ]:
X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]

**70:30 data split with stratification**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

**Display training and test set**

In [ ]:
print("Ukuran set pelatihan:", X_train.shape)
print("Ukuran set pengujian:", X_test.shape)

Ukuran set pelatihan: (761, 881)
Ukuran set pengujian: (327, 881)


In [ ]:
print("Distribusi kelas pada set pelatihan:")
print(y_train.value_counts())

print("\nDistribusi kelas pada set pengujian:")
print(y_test.value_counts())

Distribusi kelas pada set pelatihan:
Eye disorders
1    481
0    280
Name: count, dtype: int64

Distribusi kelas pada set pengujian:
Eye disorders
1    206
0    121
Name: count, dtype: int64


**Perform Random OverSampler (ROS) to balance the training set**

In [ ]:
ros = RandomOverSampler(random_state=42)
X_train_sampling, y_train_sampling = ros.fit_resample(X_train, y_train)

In [ ]:
print("\nDistribusi kelas setelah ROS pada set pelatihan:")
print(y_train_sampling.value_counts())


Distribusi kelas setelah ROS pada set pelatihan:
Eye disorders
1    481
0    481
Name: count, dtype: int64


**Save train and test sets into a csv file**

In [ ]:
X_train_sampling.to_csv('X_train_smilesdataset.csv', index=False)
y_train_sampling.to_csv('y_train_smilesdataset.csv', index=False)
X_test.to_csv('X_test_smilesdataset.csv', index=False)
y_test.to_csv('y_test_smilesdataset.csv', index=False)

**Download csv files**

In [ ]:
from google.colab import files

files.download('X_train_smilesdataset.csv')
files.download('y_train_smilesdataset.csv')
files.download('X_test_smilesdataset.csv')
files.download('y_test_smilesdataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Variance Threshold

**VarianceThreshold object initialisation**

In [ ]:
threshold = 0.1
selector = VarianceThreshold(threshold)

**Feature selection on training data**

In [ ]:
X_selected = selector.fit_transform(X_train)
features_result = selector.get_feature_names_out()

**Display feature selection result information**

In [ ]:
print('Jumlah fitur awal:', X_train.shape[1])
print('Jumlah fitur terpilih setelah seleksi:', features_result.shape[0])

Jumlah fitur awal: 881
Jumlah fitur terpilih setelah seleksi: 256


**Print selected features**

In [ ]:
print("Fitur yang dipilih setelah VarianceThreshold:", features_result)

Fitur yang dipilih setelah VarianceThreshold: ['PubchemFP2' 'PubchemFP11' 'PubchemFP12' 'PubchemFP14' 'PubchemFP15'
 'PubchemFP16' 'PubchemFP18' 'PubchemFP19' 'PubchemFP20' 'PubchemFP23'
 'PubchemFP33' 'PubchemFP37' 'PubchemFP143' 'PubchemFP145' 'PubchemFP146'
 'PubchemFP178' 'PubchemFP179' 'PubchemFP180' 'PubchemFP181'
 'PubchemFP185' 'PubchemFP186' 'PubchemFP192' 'PubchemFP255'
 'PubchemFP256' 'PubchemFP257' 'PubchemFP259' 'PubchemFP285'
 'PubchemFP286' 'PubchemFP287' 'PubchemFP293' 'PubchemFP294'
 'PubchemFP299' 'PubchemFP308' 'PubchemFP333' 'PubchemFP334'
 'PubchemFP335' 'PubchemFP338' 'PubchemFP339' 'PubchemFP340'
 'PubchemFP341' 'PubchemFP342' 'PubchemFP345' 'PubchemFP346'
 'PubchemFP351' 'PubchemFP352' 'PubchemFP353' 'PubchemFP355'
 'PubchemFP356' 'PubchemFP357' 'PubchemFP358' 'PubchemFP362'
 'PubchemFP365' 'PubchemFP366' 'PubchemFP370' 'PubchemFP371'
 'PubchemFP372' 'PubchemFP373' 'PubchemFP374' 'PubchemFP375'
 'PubchemFP376' 'PubchemFP377' 'PubchemFP379' 'PubchemFP380'
 'Pubch

# Save & Download in Pickle File

In [ ]:
import pickle

with open('VTresult.pkl', 'wb') as pickle_file:
    pickle.dump(features_result, pickle_file)

print('Output hasil Variance Threshold berhasil disimpan ke VTresult.pkl')

from google.colab import files
files.download('VTresult.pkl')

Output hasil Variance Threshold berhasil disimpan ke VTresult.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>